# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [78]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [79]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hede,33.7681,120.2517,29.88,79,0,3.97,CN,1722383351
1,1,edinburgh of the seven seas,-37.0676,-12.3116,15.09,90,80,16.47,SH,1722383354
2,2,horta,38.5333,-28.6333,21.68,84,77,3.13,PT,1722383358
3,3,jamestown,42.0970,-79.2353,24.77,91,75,2.57,US,1722383362
4,4,grytviken,-54.2811,-36.5092,-0.94,96,87,5.90,GS,1722383365


In [80]:
city_data_df["Max Fahrenheit"] = city_data_df["Max Temp"] * 9/5 +32
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Max Fahrenheit
0,0,hede,33.7681,120.2517,29.88,79,0,3.97,CN,1722383351,85.784
1,1,edinburgh of the seven seas,-37.0676,-12.3116,15.09,90,80,16.47,SH,1722383354,59.162
2,2,horta,38.5333,-28.6333,21.68,84,77,3.13,PT,1722383358,71.024
3,3,jamestown,42.0970,-79.2353,24.77,91,75,2.57,US,1722383362,76.586
4,4,grytviken,-54.2811,-36.5092,-0.94,96,87,5.90,GS,1722383365,30.308


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [81]:
#%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x = "Lng",
    y = "Lat",
    s = "Humidity",
    c = "City",
    geo = True,
    tiles = "OSM", 
    width = 1000,
    height = 550,
    alpha = 0.5,
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [82]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df.loc[
    (city_data_df["Max Fahrenheit"] < 90) &
    (city_data_df["Max Fahrenheit"] > 75) &
    (city_data_df["Wind Speed"] < 10) &
    (city_data_df["Cloudiness"] <5)
].dropna()

print(len(ideal_cities_df))
# Display sample data
ideal_cities_df.head()

36


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Max Fahrenheit
0,0,hede,33.7681,120.2517,29.88,79,0,3.97,CN,1722383351,85.784
9,9,siwa oasis,29.2041,25.5195,30.06,40,0,3.82,EG,1722383384,86.108
38,38,huilong,31.8111,121.6550,29.26,79,0,5.47,CN,1722383487,84.668
76,76,el granada,37.5027,-122.4694,25.82,71,0,7.72,US,1722383624,78.476
101,101,touros,-5.1989,-35.4608,24.35,84,4,6.04,BR,1722383713,75.830


### Step 3: Create a new DataFrame called `hotel_df`.

In [83]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,hede,CN,33.7681,120.2517,79,
9,siwa oasis,EG,29.2041,25.5195,40,
38,huilong,CN,31.8111,121.6550,79,
76,el granada,US,37.5027,-122.4694,71,
101,touros,BR,-5.1989,-35.4608,84,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [84]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key, 
    "categories": "accommodation.hotel"   
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hede - nearest hotel: No hotel found
siwa oasis - nearest hotel: Flower of the city hotel
huilong - nearest hotel: 先豪国际酒店
el granada - nearest hotel: Beach House
touros - nearest hotel: Pousada Atlântico
half moon bay - nearest hotel: Half Moon Bay Inn
pozoblanco - nearest hotel: Casa Rural Abuela Teo
kill devil hills - nearest hotel: Mariner Days Inn & Suites
sovet - nearest hotel: No hotel found
zhanaozen - nearest hotel: No hotel found
santa eularia des riu - nearest hotel: Hostal Sa Rota
daimiel - nearest hotel: Hotel Doña Manuela
myrina - nearest hotel: Ήφαιστος
archangelos - nearest hotel: Semina
sao felix do xingu - nearest hotel: Hotel Terraço
amarante do maranhao - nearest hotel: No hotel found
urucui - nearest hotel: Formula Flat hotel
aripuana - nearest hotel: No hotel found
portland - nearest hotel: the Hoxton
atamyrat - nearest hotel: hotel housing working
riberalta - nearest hotel: Hotel Jomali
kongolo - nearest hotel: No hotel found
paphos - nearest

,City,Country,Lat,Lng,Humidity,Hotel Name
0,hede,CN,33.7681,120.2517,79,No hotel found
9,siwa oasis,EG,29.2041,25.5195,40,Flower of the city hotel
38,huilong,CN,31.8111,121.6550,79,先豪国际酒店
76,el granada,US,37.5027,-122.4694,71,Beach House
101,touros,BR,-5.1989,-35.4608,84,Pousada Atlântico
130,half moon bay,US,37.4636,-122.4286,69,Half Moon Bay Inn
142,pozoblanco,ES,38.3791,-4.8483,36,Casa Rural Abuela Teo
151,kill devil hills,US,36.0307,-75.6760,79,Mariner Days Inn & Suites
216,sovet,TJ,38.0467,69.5882,27,No hotel found
230,zhanaozen,KZ,43.3000,52.8000,31,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [85]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    x = "Lng",
    y = "Lat",
    s = "Humidity",
    c = "City",
    geo = True,
    tiles = "OSM", 
    width = 1000,
    height = 550,
    alpha = 0.5,
    hover_cols = ["Country", "Hotel Name"],
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)